In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')

In [6]:
dataset = pd.read_csv('Stats.csv')

In [10]:
dataset_imp.describe()

,4xxErrorRate,5xxErrorRate,Requests,TotalErrorRate
count,141744.000000,141744.000000,141744.000000,141744.000000
mean,0.031709,0.001010,75.089725,0.032719
std,1.100194,0.043898,247.768057,1.101040
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,100.000000,9.458023,1760.000000,100.000000


In [11]:
#creating dummies
df = pd.concat([dataset_imp,pd.get_dummies(dataset_imp['CNAMES'], prefix='CNAMES')],axis=1)
df.drop(['CNAMES'],axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(dataset_imp['DOMAIN_NAME'], prefix='DOMAIN_NAME')],axis=1)
df.drop(['DOMAIN_NAME'],axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(dataset_imp['DistributionId'], prefix='DistributionId')],axis=1)
df.drop(['DistributionId'],axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(dataset_imp['ORIGIN'], prefix='ORIGIN')],axis=1)
df.drop(['ORIGIN'],axis=1, inplace=True)

In [13]:
df=df[[c for c in df if c not in ['TotalErrorRate']] + ['TotalErrorRate']]

In [ ]:
#pair plot wrt target column i.e. 'TotalErrorRate'
sns.pairplot(data=df, y_vars=df.iloc[:, :-1].columns.tolist(), x_vars=['TotalErrorRate'])

In [19]:
#correlation matrix
corrmat = df.corr()

In [ ]:
#top 10(value of 'k') correlated features wrt 'TotalErrorRate'
k=10
cols = corrmat.nlargest(k, 'TotalErrorRate')['TotalErrorRate']
print(cols)

In [ ]:
#heatmap-top 10(value of 'k') correlated features wrt 'TotalErrorRate'
cols = corrmat.nlargest(k, 'TotalErrorRate')['TotalErrorRate'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [22]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

In [28]:
scaled=sc_X.fit_transform(df)

In [30]:
df_scaled = pd.DataFrame(data=scaled,columns=df.columns.tolist())

In [31]:
#correlation matrix
corrmat = df_scaled.corr()

In [ ]:
k=10
#heatmap-top 10(value of 'k') correlated features wrt 'TotalErrorRate'
cols = corrmat.nlargest(k, 'TotalErrorRate')['TotalErrorRate'].index
cm = np.corrcoef(df_scaled[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [35]:
#chi-2 test
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [36]:
#features
X = df.iloc[:, :-1]
#target column
Y= df.iloc[:, 23]

In [37]:
selector = SelectKBest(chi2, k=5)

In [38]:
y = np.asarray(Y, dtype="int")
print(type(y))

<class 'numpy.ndarray'>


In [39]:
#top 5 (value of 'k') features by chi-2 test
selector = SelectKBest(chi2, k=5)
selector.fit(X, y)
#Selected_feature_names=X.columns[selector.get_support()]

SelectKBest(k=5, score_func=<function chi2 at 0x000001A6E7706158>)

In [40]:
Selected_feature_names=X.columns[selector.get_support()]

In [44]:
print('Top features by chi-2 test')
for f in Selected_feature_names:
    print(f)

Top features by chi-2 test
Requests
CNAMES_auth.ti-platform.com
DOMAIN_NAME_dcjh1cybvrnys.cloudfront.net
DistributionId_E1O4MVYDROXZ8A
ORIGIN_auth.ti-platform.com.s3.amazonaws.com


In [45]:
from sklearn.feature_selection import f_classif

In [46]:
#ANOVA F-value For Feature Selection
# Create an SelectKBest object to select features with 5 best ANOVA F-Values
selector = SelectKBest(f_classif, k=5)

In [47]:
selector.fit(X,y )

SelectKBest(k=5, score_func=<function f_classif at 0x000001A6E7706048>)

In [48]:
Selected_feature_names=X.columns[selector.get_support()]

In [49]:
print('Top features by ANOVA test')
for f in Selected_feature_names:
    print(f)

Top features by ANOVA test
CNAMES_auth.ti-platform.com
DOMAIN_NAME_dcjh1cybvrnys.cloudfront.net
DistributionId_E1O4MVYDROXZ8A
ORIGIN_auth.ti-platform.com.s3.amazonaws.com
ORIGIN_ti-prod-emeacacheelb-1779963629.eu-west-1.elb.amazonaws.com
